# 1 - Setup

In [1]:
# imports from Python standard library

# imports requiring installation
#   connection to Google Cloud Storage
from google.cloud import storage            # pip install google-cloud-storage
from google.oauth2 import service_account   # pip install google-auth

#  data science packages
import numpy as np                          # pip install numpy
import pandas as pd                         # pip install pandas

In [2]:
# imports from tweet_turing.py
import tweet_turing as tur      # note - different import approach from prior notebooks

# imports from tweet_turing_paths.py
from tweet_turing_paths import local_data_paths, local_snapshot_paths, gcp_data_paths, \
    gcp_snapshot_paths, gcp_project_name, gcp_bucket_name, gcp_key_file

In [3]:
# pandas options
pd.set_option('display.max_colwidth', None)

## Local or Cloud?

Decide here whether to run notebook with local data or GCP bucket data
 - if the working directory of this notebook has a "../data/" folder with data loaded (e.g. working on local computer or have data files loaded to a cloud VM) then use the "local files" option and comment out the "gcp bucket files" option
 - if this notebook is being run from a GCP VM (preferrably in the `us-central1` location) then use the "gcp bucket files" option and comment out the "local files" option

In [4]:
# option: local files
local_or_cloud: str = "local"   # comment/uncomment this line or next

# option: gcp bucket files
#local_or_cloud: str = "cloud"   # comment/uncomment this line or previous

# don't comment/uncomment for remainder of cell
if (local_or_cloud == "local"):
    data_paths = local_data_paths
    snapshot_paths = local_snapshot_paths
elif (local_or_cloud == "cloud"):
    data_paths = gcp_data_paths
    snapshot_paths = gcp_snapshot_paths
else:
    raise ValueError("Variable 'local_or_cloud' can only take on one of two values, 'local' or 'cloud'.")
    # subsequent cells will not do this final "else" check

In [5]:
# this cell only needs to run its code if local_or_cloud=="cloud"
#   (though it is harmless if run when local_or_cloud=="local")
gcp_storage_client: storage.Client = None
gcp_bucket: storage.Bucket = None

if (local_or_cloud == "cloud"):
    gcp_storage_client = tur.get_gcp_storage_client(project_name=gcp_project_name, key_file=gcp_key_file)
    gcp_bucket = tur.get_gcp_bucket(storage_client=gcp_storage_client, bucket_name=gcp_bucket_name)

In [6]:
# note this cell requires package `pyarrow` to be installed in environment
parq_filename: str = "data_after_03_eda.parquet.gz"
parq_path: str = f"{snapshot_paths['parq_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    df_full = pd.read_parquet(parq_path, engine='pyarrow')
elif (local_or_cloud == "cloud"):
    pass    # TODO: implement loading of cloud file

# BERT With Tabular Data 

In [7]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'

c:\Users\Johns\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Create a dataframe only containing tweets and the class
#Random sample of 8k
bert_df = df_full.sample(n=8000, random_state=1)

In [9]:
bert_df.head()

,external_author_id,author,content,region,language,following,followers,updates,post_type,is_retweet,...,tco1_step1,data_source,has_url,emoji_text,emoji_count,publish_date,class,following_ratio,class_numeric,RUS_lett_count
1459252,3252787160,FUNDDIET,http://t.co/ZUs02Wqv2c Rooftop 5 movables so exercise phuket emT @geeoharee @apol___ @Llydisblur @LaylaRalphs @RachelAucker @Billygibson94,United States,en,5,347,12844,NaN,0.0,...,https://twitter.com/safety/unsafe_link_warning?unsafe_link=http%3A%2F%2Fwww.LoseFat.pw%2FRooftop--movables-so-exercise-phuket-emT.html,Troll,1,[],0,2015-07-11 19:52:00+00:00,Troll,0.014368,1,0
2711438,115430404,dpbrugler,"@215khalil 6-0, 225. We'll see what the Combine says.",Northeast Ohio,en,842,135515,1,replied_to,0.0,...,<NA>,verified_random,0,[],0,2017-12-03 01:24:51+00:00,Verified,0.006213,0,0
2052811,902000000000000000,PAMELA_SHARKY13,Toxic masculinity and privilege. https://t.co/EE9xVit4d4,United States,en,1305,587,64,RETWEET,1.0,...,https://twitter.com/ABC/status/901907098325352448/photo/1,Troll,1,[],0,2017-08-28 01:28:00+00:00,Troll,2.219388,1,0
3556050,208775627,B_McGee32,RT @MrMcGee33: Yessirrr headed to Hard Rock with my bro @B_McGee21 #Vibin #Family #Brothers,FLL/NYC/LA,en,2522,8203,1,retweeted,1.0,...,<NA>,verified_random,0,[],0,2013-07-06 02:44:33+00:00,Verified,0.307411,0,0
1188826,2494112058,DAILYSANJOSE,#SanJose Oakland Singer Kehlani Survives Suicide Attempt,United States,en,4282,16829,15573,NaN,0.0,...,<NA>,Troll,0,[],0,2016-03-29 23:04:00+00:00,Troll,0.254427,1,0


In [10]:
train_df, val_df, test_df = np.split(bert_df.sample(frac=1), [int(.8*len(bert_df)), int(.9 * len(bert_df))])
print('Num examples train-val-test')
print(len(train_df), len(val_df), len(test_df))
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

Num examples train-val-test
6400 800 800


# Experiment Parameters

In [11]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
  numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

In [12]:
#Define our data types 
text_cols = ['content']

#For easy model I picked out specific columns. Change accordingly!!!
categorical_cols = ['post_type']
numerical_cols = ['has_url','emoji_count', 'following_ratio', 'following','followers']


#### #We can use below code for large mass of features. Just keep in mind to remove the account category, data_source, class and class numeric
# categorical_cols = bert_df.select_dtypes(include='category')
# numerical_cols = bert_df.select_dtypes(include=['int64','float64','uint64'])

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': categorical_cols,
    'label_col': 'class_numeric',
    'label_list': ['Troll', 'Verified']
}


model_args = ModelArguments(
    model_name_or_path='distilbert-base-uncased-finetuned-sst-2-english'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='weighted_feature_sum_on_transformer_cat_and_numerical_feats',
    column_info=column_info_dict,
    task='classification'
)

#Default 
# data_args = MultimodalDataTrainingArguments(
#     data_path='.',
#     combine_feat_method='gating_on_cat_and_num_feats_then_sum',
#     column_info=column_info_dict,
#     task='classification'
# )

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    warmup_steps=500,   
    num_train_epochs=1,
    weight_decay=0.01, 
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=30

)

#Default Args 
# training_args = TrainingArguments(
#     output_dir="./logs/model_name",
#     logging_dir="./logs/runs",
#     overwrite_output_dir=True,
#     do_train=True,
#     do_eval=True,
#     per_device_train_batch_size=32,
#     num_train_epochs=1,
#     evaluate_during_training=True,
#     logging_steps=25,
#     eval_steps=250
# )

set_seed(training_args.seed)

In [13]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  distilbert-base-uncased-finetuned-sst-2-english


In [14]:
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

c:\Users\Johns\AppData\Local\Programs\Python\Python38\lib\site-packages\multimodal_transformers\data\data_utils.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = df[num_cols].fillna(df[num_cols].median())
c:\Users\Johns\AppData\Local\Programs\Python\Python38\lib\site-packages\multimodal_transformers\data\data_utils.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = df[num_cols].fillna(df[num_cols].median())
Asking to truncate to max_length but no maximum length is provid

In [15]:
num_labels = len(np.unique(train_dataset.labels))
num_labels

2

In [16]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [17]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of DistilBertWithTabular were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['tabular_combiner.weight_cat', 'tabular_combiner.weight_num', 'tabular_combiner.num_bn.weight', 'tabular_combiner.num_bn.bias', 'tabular_combiner.num_bn.running_mean', 'tabular_combiner.num_bn.running_var', 'tabular_combiner.cat_layer.weight', 'tabular_combiner.cat_layer.bias', 'tabular_combiner.num_layer.weight', 'tabular_combiner.num_layer.bias', 'tabular_combiner.layer_norm.weight', 'tabular_combiner.layer_norm.bias', 'tabular_classifier.weight', 'tabular_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

In [19]:
def calc_classification_metrics(p: EvalPrediction):
  pred_labels = np.argmax(p.predictions, axis=1)
  pred_scores = softmax(p.predictions, axis=1)[:, 1]
  labels = p.label_ids
  if len(np.unique(labels)) == 2:  # binary classification
      roc_auc_pred_score = roc_auc_score(labels, pred_scores)
      precisions, recalls, thresholds = precision_recall_curve(labels,
                                                                pred_scores)
      fscore = (2 * precisions * recalls) / (precisions + recalls)
      fscore[np.isnan(fscore)] = 0
      ix = np.argmax(fscore)
      threshold = thresholds[ix].item()
      pr_auc = auc(recalls, precisions)
      tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
      result = {'roc_auc': roc_auc_pred_score,
                'threshold': threshold,
                'pr_auc': pr_auc,
                'recall': recalls[ix].item(),
                'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
                'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
                }
  else:
      acc = (pred_labels == labels).mean()
      f1 = f1_score(y_true=labels, y_pred=pred_labels)
      result = {
          "acc": acc,
          "f1": f1,
          "acc_and_f1": (acc + f1) / 2,
          "mcc": matthews_corrcoef(labels, pred_labels)
      }

  return result

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)

In [21]:
%%time
trainer.train()

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

{'loss': 0.6647732543945313, 'learning_rate': 2.5e-06, 'epoch': 0.03125, 'step': 25}







































































































Evaluation: 100%|██████████| 100/100 [00:33<00:00,  2.94it/s]


{'eval_loss': 0.6497141176462173, 'eval_roc_auc': 0.7498064359254922, 'eval_threshold': 0.3665297329425812, 'eval_pr_auc': 0.80073216574209, 'eval_recall': 0.9392624728850325, 'eval_precision': 0.6744548286604362, 'eval_f1': 0.7851314596554851, 'eval_tn': 291, 'eval_fp': 48, 'eval_fn': 265, 'eval_tp': 196, 'epoch': 0.0375, 'step': 30}


{'loss': 0.6794577026367188, 'learning_rate': 5e-06, 'epoch': 0.0625, 'step': 50}


...






































































































Evaluation: 100%|██████████| 100/100 [00:34<00:00,  2.86it/s]


{'eval_loss': 0.1369975937018171, 'eval_roc_auc': 0.9899282693132154, 'eval_threshold': 0.529875636100769, 'eval_pr_auc': 0.9926170457407513, 'eval_recall': 0.9804772234273319, 'eval_precision': 0.9475890985324947, 'eval_f1': 0.9637526652452025, 'eval_tn': 308, 'eval_fp': 31, 'eval_fn': 8, 'eval_tp': 453, 'epoch': 0.975, 'step': 780}


Epoch: 100%|██████████| 1/1 [42:07<00:00, 2527.80s/it]

{'loss': 0.186212158203125, 'learning_rate': 0.0, 'epoch': 1.0, 'step': 800}
CPU times: total: 4h 11min 1s
Wall time: 42min 7s


TrainOutput(global_step=800, training_loss=0.3399104690551758)

In [22]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [23]:
%tensorboard --logdir ./logs/runs --port=6006